In [4]:
import modin.pandas as pd
import numpy as np
import numpy as np
import optuna

import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
import xgboost as xgb


test_df = pd.read_csv(r'test_dataset.csv')
train_df = pd.read_csv(r'balanced_train_dataset.csv')




label = ['Analysis','Backdoor','DoS','Exploits','Fuzzers','Generic','Normal','Reconnaissance','Shellcode','Worms']

X_train = train_df.iloc[:,:train_df.shape[1]-10]
Y_train = train_df.iloc[:,-10:].to_numpy()

X_test = test_df.iloc[:,:test_df.shape[1]-10]
Y_test = test_df.iloc[:,-10:].to_numpy()



def LabelAttack(Y):
  attack = []
  for i in range(Y.shape[0]):
    k = 0
    for j in range(Y.shape[1]):
      if(Y[i][j] == 1):
        k = j
        break
    attack.append(k)
  return attack


Y_train = np.asarray(LabelAttack(Y_train)).reshape(-1,1)
Y_test = np.asarray(LabelAttack(Y_test)).reshape(-1,1)


In [9]:
params = {
    "booster": "dart",
    "verbosity": 2,
    "objective": "multi:softmax",
    "num_class" : 10,
    "lambda": 1.234568712743763e-06,
    "alpha": 0.021824183515918392,
    "subsample": 0.7966629501270384,
    "colsample_bytree": 0.8575214799710436,
    "early_stopping_rounds": 24,
    "n_estimators": 32,
    "max_depth": 7,
    "min_child_weight": 5,
    "eta": 0.020721025441133932,
    "gamma": 8.632145831151602e-05,
    "grow_policy": "depthwise",
    "sample_type": "uniform",
    "normalize_type": "forest",
    "rate_drop": 1.4263688272813651e-08,
    "skip_drop": 3.191224113185437e-05
}

In [10]:

import xgboost as xgb

dtrain = xgb.DMatrix(X_train, label=Y_train)
dvalid = xgb.DMatrix(X_test, label=Y_test)  


bst = xgb.train(params, dtrain)
preds = bst.predict(dvalid)
pred_labels = np.rint(preds)
accuracy = sklearn.metrics.accuracy_score(Y_test, pred_labels)
accuracy

[19:59:57] WARNING: ../src/learner.cc:573: 
Parameters: { "early_stopping_rounds", "n_estimators" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[19:59:58] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 226 extra nodes, 0 pruned nodes, max_depth=7
[19:59:58] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 216 extra nodes, 0 pruned nodes, max_depth=7
[19:59:59] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 172 extra nodes, 0 pruned nodes, max_depth=7
[19:59:59] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 172 extra nodes, 0 pruned nodes, max_depth=7
[20:00:00] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 182 extra nodes, 0 pruned nodes, max_depth=7
[20:00:00] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 64

[20:00:31] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 176 extra nodes, 0 pruned nodes, max_depth=7
[20:00:31] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 64 extra nodes, 0 pruned nodes, max_depth=7
[20:00:32] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=5
[20:00:32] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 230 extra nodes, 0 pruned nodes, max_depth=7
[20:00:33] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 234 extra nodes, 0 pruned nodes, max_depth=7
[20:00:34] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 220 extra nodes, 0 pruned nodes, max_depth=7
[20:00:34] INFO: ../src/gbm/gbtree.cc:860: drop 0 trees, weight = 1
[20:00:36] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 224 extra nodes, 0 pruned nodes, max_depth=7
[20:00:37] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 222 extra nodes, 0 pruned nodes, max_depth=7
[20:00:37] INFO: ../sr

0.869953359568576

In [3]:
from xgboost import XGBClassifier

model = XGBClassifier(params)
    
model.fit(X_train, Y_train)
preds = model.predict(X_test)
pred_labels = np.rint(preds)
accuracy = sklearn.metrics.accuracy_score(Y_test, pred_labels)

accuracy

[17:21:14] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


NameError: name 'sklearn' is not defined

In [4]:
import sklearn

In [5]:
accuracy = sklearn.metrics.accuracy_score(Y_test, pred_labels)

accuracy

0.771838410338629

In [10]:
"""
Optuna example that optimizes a classifier configuration for cancer dataset
using XGBoost.
In this example, we optimize the validation accuracy of cancer detection
using XGBoost. We optimize both the choice of booster model and its
hyperparameters.
"""

import numpy as np
import optuna

import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
import xgboost as xgb


def objective(trial):

    dtrain = xgb.DMatrix(X_train, label=Y_train)
    dvalid = xgb.DMatrix(X_test, label=Y_test)

    param = {
        "verbosity": 2,
        "objective": "multi:softmax",
        # use exact for small dataset.
        #"tree_method": "exact",
        "num_class" : 10,
        # defines booster, gblinear for linear functions.
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        # L2 regularization weight.
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        # L1 regularization weight.
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        # sampling ratio for training data.
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        # sampling according to each tree.
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
        "early_stopping_rounds": trial.suggest_int("early_stopping_rounds", 8, 32),
        "n_estimators": trial.suggest_categorical("n_estimators", [16, 32, 64, 96]),
    }

    if param["booster"] in ["gbtree", "dart"]:
        # maximum depth of the tree, signifies complexity of the tree.
        param["max_depth"] = trial.suggest_int("max_depth", 3, 9, step=2)
        # minimum child weight, larger the term more conservative the tree.
        param["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        # defines how selective algorithm is.
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    bst = xgb.train(param, dtrain)
    preds = bst.predict(dvalid)
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.accuracy_score(Y_test, pred_labels)
    return accuracy


if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100, timeout=600)

    print("Number of finished trials: ", len(study.trials))
    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))
    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2023-01-15 16:48:00,272] A new study created in memory with name: no-name-4629fd78-abe2-414a-9148-4d373f7e2915


[16:48:00] WARNING: ../src/learner.cc:573: 
Parameters: { "early_stopping_rounds", "n_estimators" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[16:48:01] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=3
[16:48:01] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=3
[16:48:01] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=3
[16:48:01] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=3
[16:48:01] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=3
[16:48:02] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 14 extr

[16:48:09] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 12 extra nodes, 0 pruned nodes, max_depth=3
[16:48:09] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=3
[16:48:09] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=3
[16:48:10] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=3
[16:48:10] INFO: ../src/gbm/gbtree.cc:860: drop 21 trees, weight = 0.047619
[16:48:10] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=3
[16:48:10] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=3
[16:48:10] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=3
[16:48:11] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=3
[16:48:11] INFO: ../

[I 2023-01-15 16:48:15,194] Trial 0 finished with value: 0.7571661079531652 and parameters: {'booster': 'dart', 'lambda': 2.3620553438308152e-07, 'alpha': 0.0006228471086087204, 'subsample': 0.6998629730806571, 'colsample_bytree': 0.26939862938271997, 'early_stopping_rounds': 21, 'n_estimators': 16, 'max_depth': 3, 'min_child_weight': 3, 'eta': 2.620558962123999e-06, 'gamma': 1.3426228619144058e-06, 'grow_policy': 'lossguide', 'sample_type': 'uniform', 'normalize_type': 'tree', 'rate_drop': 0.32893317446988873, 'skip_drop': 0.05509819097652605}. Best is trial 0 with value: 0.7571661079531652.


[16:48:15] WARNING: ../src/learner.cc:573: 
Parameters: { "colsample_bytree", "early_stopping_rounds", "n_estimators", "subsample" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[16:48:15] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softmax' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


...

[I 2023-01-15 16:56:41,576] Trial 12 finished with value: 0.8566778409366953 and parameters: {'booster': 'dart', 'lambda': 0.009017823471063138, 'alpha': 1.1389549873446476e-05, 'subsample': 0.4323591972929229, 'colsample_bytree': 0.81943191099166, 'early_stopping_rounds': 32, 'n_estimators': 16, 'max_depth': 7, 'min_child_weight': 8, 'eta': 1.8490015381076588e-05, 'gamma': 0.8843105180993651, 'grow_policy': 'depthwise', 'sample_type': 'weighted', 'normalize_type': 'forest', 'rate_drop': 1.6239907087915965e-05, 'skip_drop': 2.4275606282485565e-06}. Best is trial 4 with value: 0.8656536947966769.


[16:56:42] WARNING: ../src/learner.cc:573: 
Parameters: { "early_stopping_rounds", "n_estimators" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[16:56:43] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 226 extra nodes, 0 pruned nodes, max_depth=7
[16:56:44] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 216 extra nodes, 0 pruned nodes, max_depth=7
[16:56:44] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 172 extra nodes, 0 pruned nodes, max_depth=7
[16:56:45] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 172 extra nodes, 0 pruned nodes, max_depth=7
[16:56:45] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 182 extra nodes, 0 pruned nodes, max_depth=7
[16:56:46] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 64

[16:57:27] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 176 extra nodes, 0 pruned nodes, max_depth=7
[16:57:28] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 64 extra nodes, 0 pruned nodes, max_depth=7
[16:57:28] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 14 extra nodes, 0 pruned nodes, max_depth=5
[16:57:29] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 230 extra nodes, 0 pruned nodes, max_depth=7
[16:57:30] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 234 extra nodes, 0 pruned nodes, max_depth=7
[16:57:31] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 220 extra nodes, 0 pruned nodes, max_depth=7
[16:57:31] INFO: ../src/gbm/gbtree.cc:860: drop 0 trees, weight = 1
[16:57:33] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 224 extra nodes, 0 pruned nodes, max_depth=7
[16:57:34] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 222 extra nodes, 0 pruned nodes, max_depth=7
[16:57:36] INFO: ../sr

[I 2023-01-15 16:58:03,250] Trial 13 finished with value: 0.869953359568576 and parameters: {'booster': 'dart', 'lambda': 1.234568712743763e-06, 'alpha': 0.021824183515918392, 'subsample': 0.7966629501270384, 'colsample_bytree': 0.8575214799710436, 'early_stopping_rounds': 24, 'n_estimators': 32, 'max_depth': 7, 'min_child_weight': 5, 'eta': 0.020721025441133932, 'gamma': 8.632145831151602e-05, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'forest', 'rate_drop': 1.4263688272813651e-08, 'skip_drop': 3.191224113185437e-05}. Best is trial 13 with value: 0.869953359568576.


Number of finished trials:  14
Best trial:
  Value: 0.869953359568576
  Params: 
    booster: dart
    lambda: 1.234568712743763e-06
    alpha: 0.021824183515918392
    subsample: 0.7966629501270384
    colsample_bytree: 0.8575214799710436
    early_stopping_rounds: 24
    n_estimators: 32
    max_depth: 7
    min_child_weight: 5
    eta: 0.020721025441133932
    gamma: 8.632145831151602e-05
    grow_policy: depthwise
    sample_type: uniform
    normalize_type: forest
    rate_drop: 1.4263688272813651e-08
    skip_drop: 3.191224113185437e-05


In [3]:
"""
Optuna example that optimizes a classifier configuration for cancer dataset
using XGBoost.
In this example, we optimize the validation accuracy of cancer detection
using XGBoost. We optimize both the choice of booster model and its
hyperparameters.
"""

import numpy as np
import optuna

import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
import xgboost as xgb
from xgboost import XGBClassifier


def objective(trial):

    param = {
        "verbosity": 2,
        #"objective": "binary:logistic",
        # use exact for small dataset.
        #"tree_method": "exact",
        # defines booster, gblinear for linear functions.
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        # L2 regularization weight.
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        # L1 regularization weight.
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        # sampling ratio for training data.
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        # sampling according to each tree.
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
        "early_stopping_rounds": trial.suggest_int("early_stopping_rounds", 8, 32),
        "n_estimators": trial.suggest_categorical("n_estimators", [16, 32, 64, 96]),
    }

    if param["booster"] in ["gbtree", "dart"]:
        # maximum depth of the tree, signifies complexity of the tree.
        param["max_depth"] = trial.suggest_int("max_depth", 3, 9, step=2)
        # minimum child weight, larger the term more conservative the tree.
        param["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        # defines how selective algorithm is.
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    model = XGBClassifier(param)
    
    model.fit(X_train, Y_train)
    preds = model.predict(X_test)
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.accuracy_score(Y_test, pred_labels)
    return accuracy


if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100, timeout=600)

    print("Number of finished trials: ", len(study.trials))
    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))
    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2023-01-14 23:40:33,260] A new study created in memory with name: no-name-9affed5f-f299-46f8-aaac-257df3dc2349


[23:41:01] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


KeyboardInterrupt: 

In [26]:
train_df.shape, X_train.shape

((363144, 54), (363144, 44))

In [30]:
df = pd.DataFrame(X_train)
df['label'] = Y_train
df

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,label
0,0.000000,0.002025,0.856061,0.090909,0.400000,4.697041e-04,3.630423e-04,1.630009e-05,1.173458e-05,7.408749e-05,...,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,6
1,0.000006,0.010832,0.856061,0.090909,0.400000,1.221231e-03,3.448902e-03,4.945754e-05,2.866376e-03,7.847337e-05,...,0.000000e+00,0.000000e+00,0.015625,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.081967,0.000000e+00,6
2,0.000011,0.027052,0.856061,0.090909,0.400000,6.575857e-04,1.452169e-03,2.368389e-05,8.996058e-04,1.417016e-05,...,0.000000e+00,0.000000e+00,0.031250,0.000000e+00,0.000000e+00,0.000000e+00,0.016949,0.081967,0.000000e+00,6
3,0.000017,0.028027,0.856061,0.181818,0.400000,1.033349e-03,1.089127e-03,4.207373e-05,5.253272e-05,1.367711e-05,...,0.000000e+00,0.000000e+00,0.031250,2.500000e-01,2.500000e-01,0.000000e+00,0.016949,0.000000,0.000000e+00,6
4,0.000023,0.007491,0.856061,0.090909,0.400000,8.454674e-04,5.445635e-04,3.552583e-05,1.828412e-05,3.337382e-05,...,1.724138e-02,0.000000e+00,0.609375,0.000000e+00,0.000000e+00,0.000000e+00,0.016949,0.622951,0.000000e+00,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363139,0.999409,0.001643,0.789297,0.333398,0.373120,3.938322e-10,0.000000e+00,1.575631e-13,6.833961e-17,4.547849e-11,...,3.854057e-06,5.858080e-08,0.005660,5.188671e-28,4.389884e-35,1.683038e-07,0.231515,0.008128,0.000000e+00,9
363140,0.986237,0.043642,0.810754,0.398645,0.379993,2.703735e-03,5.935607e-27,4.408754e-04,9.504955e-07,3.049921e-08,...,4.883109e-05,4.198098e-05,0.008944,9.248097e-13,1.407372e-15,8.127192e-03,0.094420,0.005963,1.314299e-31,9
363141,0.989837,0.016625,0.816527,0.259905,0.398551,3.345085e-05,4.234471e-26,3.032342e-03,5.463323e-03,1.297932e-07,...,2.761655e-07,1.079367e-06,0.000182,6.375596e-12,1.560388e-16,3.913133e-04,0.027875,0.000321,2.776295e-32,9
363142,0.999992,0.000200,0.812203,0.259248,0.348079,3.870276e-13,0.000000e+00,1.221975e-18,8.655989e-25,5.934207e-18,...,4.251444e-10,1.993796e-12,0.000134,0.000000e+00,0.000000e+00,6.267140e-10,0.001202,0.000543,0.000000e+00,9


In [31]:
df = df.drop("id", axis='columns')
df

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,label
0,0.002025,0.856061,0.090909,0.400000,4.697041e-04,3.630423e-04,1.630009e-05,1.173458e-05,7.408749e-05,0.988235,...,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,6
1,0.010832,0.856061,0.090909,0.400000,1.221231e-03,3.448902e-03,4.945754e-05,2.866376e-03,7.847337e-05,0.243137,...,0.000000e+00,0.000000e+00,0.015625,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.081967,0.000000e+00,6
2,0.027052,0.856061,0.090909,0.400000,6.575857e-04,1.452169e-03,2.368389e-05,8.996058e-04,1.417016e-05,0.243137,...,0.000000e+00,0.000000e+00,0.031250,0.000000e+00,0.000000e+00,0.000000e+00,0.016949,0.081967,0.000000e+00,6
3,0.028027,0.856061,0.181818,0.400000,1.033349e-03,1.089127e-03,4.207373e-05,5.253272e-05,1.367711e-05,0.243137,...,0.000000e+00,0.000000e+00,0.031250,2.500000e-01,2.500000e-01,0.000000e+00,0.016949,0.000000,0.000000e+00,6
4,0.007491,0.856061,0.090909,0.400000,8.454674e-04,5.445635e-04,3.552583e-05,1.828412e-05,3.337382e-05,0.996078,...,1.724138e-02,0.000000e+00,0.609375,0.000000e+00,0.000000e+00,0.000000e+00,0.016949,0.622951,0.000000e+00,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363139,0.001643,0.789297,0.333398,0.373120,3.938322e-10,0.000000e+00,1.575631e-13,6.833961e-17,4.547849e-11,0.999984,...,3.854057e-06,5.858080e-08,0.005660,5.188671e-28,4.389884e-35,1.683038e-07,0.231515,0.008128,0.000000e+00,9
363140,0.043642,0.810754,0.398645,0.379993,2.703735e-03,5.935607e-27,4.408754e-04,9.504955e-07,3.049921e-08,0.981681,...,4.883109e-05,4.198098e-05,0.008944,9.248097e-13,1.407372e-15,8.127192e-03,0.094420,0.005963,1.314299e-31,9
363141,0.016625,0.816527,0.259905,0.398551,3.345085e-05,4.234471e-26,3.032342e-03,5.463323e-03,1.297932e-07,0.661793,...,2.761655e-07,1.079367e-06,0.000182,6.375596e-12,1.560388e-16,3.913133e-04,0.027875,0.000321,2.776295e-32,9
363142,0.000200,0.812203,0.259248,0.348079,3.870276e-13,0.000000e+00,1.221975e-18,8.655989e-25,5.934207e-18,1.000000,...,4.251444e-10,1.993796e-12,0.000134,0.000000e+00,0.000000e+00,6.267140e-10,0.001202,0.000543,0.000000e+00,9


In [34]:
df.to_csv("data.csv", index = False)

(deploy_ray_func pid=156446) E0115 00:15:20.608402147  168747 chttp2_transport.cc:1111]   Received a GOAWAY with error code ENHANCE_YOUR_CALM and debug data equal to "too_many_pings"


In [1]:
import pandas as pd

df = pd.read_csv("data.csv")
df

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,label
0,0.002025,0.856061,0.090909,0.400000,4.697041e-04,3.630423e-04,1.630009e-05,1.173458e-05,7.408749e-05,0.988235,...,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,6
1,0.010832,0.856061,0.090909,0.400000,1.221231e-03,3.448902e-03,4.945754e-05,2.866376e-03,7.847337e-05,0.243137,...,0.000000e+00,0.000000e+00,0.015625,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.081967,0.000000e+00,6
2,0.027052,0.856061,0.090909,0.400000,6.575857e-04,1.452169e-03,2.368389e-05,8.996058e-04,1.417016e-05,0.243137,...,0.000000e+00,0.000000e+00,0.031250,0.000000e+00,0.000000e+00,0.000000e+00,0.016949,0.081967,0.000000e+00,6
3,0.028027,0.856061,0.181818,0.400000,1.033349e-03,1.089127e-03,4.207373e-05,5.253272e-05,1.367711e-05,0.243137,...,0.000000e+00,0.000000e+00,0.031250,2.500000e-01,2.500000e-01,0.000000e+00,0.016949,0.000000,0.000000e+00,6
4,0.007491,0.856061,0.090909,0.400000,8.454674e-04,5.445635e-04,3.552583e-05,1.828412e-05,3.337382e-05,0.996078,...,1.724138e-02,0.000000e+00,0.609375,0.000000e+00,0.000000e+00,0.000000e+00,0.016949,0.622951,0.000000e+00,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363139,0.001643,0.789297,0.333398,0.373120,3.938322e-10,0.000000e+00,1.575631e-13,6.833961e-17,4.547849e-11,0.999984,...,3.854057e-06,5.858080e-08,0.005660,5.188671e-28,4.389884e-35,1.683038e-07,0.231515,0.008128,0.000000e+00,9
363140,0.043642,0.810754,0.398645,0.379993,2.703735e-03,5.935607e-27,4.408754e-04,9.504955e-07,3.049921e-08,0.981681,...,4.883109e-05,4.198098e-05,0.008944,9.248097e-13,1.407372e-15,8.127192e-03,0.094420,0.005963,1.314299e-31,9
363141,0.016625,0.816527,0.259905,0.398551,3.345085e-05,4.234471e-26,3.032342e-03,5.463323e-03,1.297932e-07,0.661793,...,2.761655e-07,1.079367e-06,0.000182,6.375596e-12,1.560388e-16,3.913133e-04,0.027875,0.000321,2.776295e-32,9
363142,0.000200,0.812203,0.259248,0.348079,3.870276e-13,0.000000e+00,1.221975e-18,8.655989e-25,5.934207e-18,1.000000,...,4.251444e-10,1.993796e-12,0.000134,0.000000e+00,0.000000e+00,6.267140e-10,0.001202,0.000543,0.000000e+00,9


In [3]:
from autogluon.tabular import TabularDataset, TabularPredictor

save_path = 'agModels-predictClass'  # specifies folder to store trained models
predictor = TabularPredictor(label='label', path=save_path).fit(df)

results = predictor.fit_summary(show_plot=True)

	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "agModels-predictClass/"
AutoGluon Version:  0.5.2
Python Version:     3.8.13
Operating System:   Linux
Train Data Rows:    363144
Train Data Columns: 42
Label Column: label
Preprocessing data ...
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == int, but few unique label-values observed).
	10 unique label values:  [6, 1, 0, 4, 8, 7, 3, 2, 9, 5]
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
/home/cse31/anaconda3/envs/automl/lib/python3.8/site-packages/autogluon/tabular/learner/default_learner.py:151: FutureWarning: iteritems 

[1000]	valid_set's multi_error: 0.122797
[2000]	valid_set's multi_error: 0.117841


	0.8835	 = Validation score   (accuracy)
	1198.12s	 = Training   runtime
	3.0s	 = Validation runtime
Fitting model: LightGBM ...


[1000]	valid_set's multi_error: 0.126927
[2000]	valid_set's multi_error: 0.125


	0.8791	 = Validation score   (accuracy)
	976.14s	 = Training   runtime
	1.86s	 = Validation runtime
Fitting model: RandomForestGini ...
	Not enough memory to train RandomForestGini... Skipping this model.
Fitting model: RandomForestEntr ...
	Not enough memory to train RandomForestEntr... Skipping this model.
Fitting model: CatBoost ...
/home/cse31/anaconda3/envs/automl/lib/python3.8/site-packages/xgboost/compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/home/cse31/anaconda3/envs/automl/lib/python3.8/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
	0.8739	 = Validation score   (accuracy)
	8976.91s	 = Training   runtime
	0

*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val  pred_time_val     fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L2   0.884086       3.116246  2437.999501                0.000933           1.123448            2       True         10
1           LightGBMXT   0.883535       3.000206  1198.116342                3.000206        1198.116342            1       True          4
2              XGBoost   0.879956       0.585033  1277.578585                0.585033        1277.578585            1       True          7
3             LightGBM   0.879130       1.857930   976.138752                1.857930         976.138752            1       True          5
4        LightGBMLarge   0.875826       0.967989   281.998076                0.967989         281.998076            1       True          9
5             CatBoost   0.873899       0.206860  8976.908802                0.206860        8976.

In [5]:
results

{'model_types': {'KNeighborsUnif': 'KNNModel',
  'KNeighborsDist': 'KNNModel',
  'NeuralNetFastAI': 'NNFastAiTabularModel',
  'LightGBMXT': 'LGBModel',
  'LightGBM': 'LGBModel',
  'CatBoost': 'CatBoostModel',
  'XGBoost': 'XGBoostModel',
  'NeuralNetTorch': 'TabularNeuralNetTorchModel',
  'LightGBMLarge': 'LGBModel',
  'WeightedEnsemble_L2': 'WeightedEnsembleModel'},
 'model_performance': {'KNeighborsUnif': 0.7348568281938326,
  'KNeighborsDist': 0.7414647577092511,
  'NeuralNetFastAI': 0.8196585903083701,
  'LightGBMXT': 0.8835352422907489,
  'LightGBM': 0.8791299559471366,
  'CatBoost': 0.873898678414097,
  'XGBoost': 0.8799559471365639,
  'NeuralNetTorch': 0.8634361233480177,
  'LightGBMLarge': 0.8758259911894273,
  'WeightedEnsemble_L2': 0.8840859030837004},
 'model_best': 'WeightedEnsemble_L2',
 'model_paths': {'KNeighborsUnif': 'agModels-predictClass/models/KNeighborsUnif/',
  'KNeighborsDist': 'agModels-predictClass/models/KNeighborsDist/',
  'NeuralNetFastAI': 'agModels-predict

In [6]:
pwd

'/home/cse31/Kaggle/Arpita/final/A=0.4'